## Classification ML project

In [ ]:
from IPython.core.display import SVG
SVG(filename='pipeline.svg')

## Loading the Data:

In [ ]:
import logistic as log
import split as spl
import least_squares as lst
import helpers as hlp
import pre_processing as pre
import vizu as viz

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import pandas as pd # cannot use external libraries, just pandas for data exploration

In [ ]:
y, tX, ids = hlp.load_csv_data(DATA_TRAIN_PATH)
_, tX_test, ids_test = hlp.load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX.shape)
print(y.shape)

## Data exploration

In [ ]:
data = pd.read_csv(DATA_TRAIN_PATH)
test_data = pd.read_csv(DATA_TEST_PATH)
dic = {'s':1,'b':-1}
data.Prediction = data.Prediction.map(dic)
test_data.Prediction = test_data.Prediction.map(dic)
data.head(10)

In [ ]:
mask = data.isin([-999]).any(axis = 1)
print(len(data[mask]))
print(len(data))

_The vast majoriy of our data has -999 values: we'd better handle it carefully_

In [ ]:
#data.replace(to_replace = -999,value = np.nan, inplace = True)

In [ ]:
#replace_val = np.nan
#tX = np.where(tX == -999,replace_val,tX)

In [ ]:
std = np.nanstd(tX,axis = 0)
mean = np.nanmean(tX,axis = 0)

In [ ]:
print('Train set size: {} samples x {} features'.format(pd.DataFrame(tX).shape[0], pd.DataFrame(tX).shape[1]))
print('Test set size: {} samples x {} features'.format(test_data.shape[0], pd.DataFrame(tX).shape[1]))

In [ ]:
data.info()

In [ ]:
data.describe()

#### Class separation - Justification

In [ ]:
col_names = list(data.columns)[2:]

data_0 = data[data['PRI_jet_num'] == 0]
data_1 = data[data['PRI_jet_num'] == 1]
data_2 = data[data['PRI_jet_num'] >= 2]

fig, axes = plt.subplots(30, 1, figsize=(8, 150), sharex=False)
for idx, name in enumerate(col_names):    
    axes[idx].hist(data_0[name], bins=50, density=True, alpha=0.3, label='class 0')
    axes[idx].hist(data_1[name], bins=50, density=True, alpha=0.3, label='class 1')
    axes[idx].hist(data_2[name], bins=50, density=True, alpha=0.3, label='class 2 & 3')
    axes[idx].set_title(name)
    axes[idx].legend()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,8))
ax0, ax1, ax2, ax3 = axes.flatten()

ax0.hist(data_0['DER_mass_transverse_met_lep'], bins=50, density=True, alpha=0.3, label='class 0', edgecolor='w', lw=.5)
ax0.hist(data_1['DER_mass_transverse_met_lep'], bins=50, density=True, alpha=0.3, label='class 1', edgecolor='w', lw=.5)
ax0.hist(data_2['DER_mass_transverse_met_lep'], bins=50, density=True, alpha=0.3, label='class 2 & 3', edgecolor='w', lw=.5)
ax0.legend()
ax0.set_xlabel('Value', fontsize=10)
ax0.set_ylabel('Proportion', fontsize=10)
ax0.set_title('DER_mass_transverse_met_lep', fontsize=12)

ax1.hist(data_0['DER_deltar_tau_lep'], bins=50, density=True, alpha=0.3, label='class 0', edgecolor='w', lw=.5)
ax1.hist(data_1['DER_deltar_tau_lep'], bins=50, density=True, alpha=0.3, label='class 1', edgecolor='w', lw=.5)
ax1.hist(data_2['DER_deltar_tau_lep'], bins=50, density=True, alpha=0.3, label='class 2 & 3', edgecolor='w', lw=.5)
ax1.legend()
ax1.set_xlabel('Value', fontsize=10)
ax1.set_ylabel('Proportion', fontsize=10)
ax1.set_title('DER_deltar_tau_lep',fontsize=12)

ax2.hist(data_0['DER_sum_pt'], bins=50, density=True, alpha=0.3, label='class 0', edgecolor='w', lw=.5)
ax2.hist(data_1['DER_sum_pt'], bins=50, density=True, alpha=0.3, label='class 1', edgecolor='w', lw=.5)
ax2.hist(data_2['DER_sum_pt'], bins=50, density=True, alpha=0.3, label='class 2 & 3', edgecolor='w', lw=.5)
ax2.legend()
ax2.set_xlabel('Value', fontsize=10)
ax2.set_ylabel('Proportion', fontsize=10)
ax2.set_title('DER_sum_pt', fontsize=12)


ax3.hist(data_0['PRI_met_sumet'], bins=50, density=True, alpha=0.3, label='class 0', edgecolor='w', lw=.5)
ax3.hist(data_1['PRI_met_sumet'], bins=50, density=True, alpha=0.3, label='class 1', edgecolor='w', lw=.5)
ax3.hist(data_2['PRI_met_sumet'], bins=50, density=True, alpha=0.3, label='class 2 & 3', edgecolor='w', lw=.5)
ax3.legend()
#ax3.ticklabel_format(useMathText=True)
#ax3.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2e'))
ax3.set_xlabel('Value', fontsize=10)
ax3.set_ylabel('Proportion', fontsize=10)
ax3.set_title('PRI_met_sumet', fontsize=12)

fig.subplots_adjust(hspace=0.3)

In [ ]:
#Save figure
fig.savefig("classes_comparison.png")

## Data Cleaning

In [ ]:
#totrash before submit: we use pandas to know to which index PRI_jet_num does correspond.
np.where(data.columns.values == "PRI_jet_num")

In [ ]:
data_trains = spl.split_categorical_data(tX,22,labels = y,split = True)
data_tests = spl.split_categorical_data(tX_test,22,split = True)

In [ ]:
mean = 0
stdev = 0
clean_data_trains = []
clean_data_tests = []
degre_polys = [12,12,13]
for i,((x_train,y_train),(x_test,test_indx)) in enumerate(zip(data_trains,data_tests)):
    x_train,x_test = pre.clean_variance(x_train,x_test)
    
    x_train = pre.clean_value(x_train,-999,np.nan)
    x_test = pre.clean_value(x_test,-999,np.nan)
    
    """
    pre.PCA_visualize(x_train,label = i)
    
    mean,eigvecs,eigvals = pre.get_PCA(x_train)
    x_test = x_test - mean
    
    x_train = pre.reduce_PCA(eigvecs,x_train,10)
    x_test = pre.reduce_PCA(eigvecs,x_test,10)
    """
    
    x_train,mean,stdev =  pre.standardize_data(x_train)
    x_test,_,_ = pre.standardize_data(x_test, mean,stdev)
    
    x_train = pre.clean_value(x_train,np.nan,0,inplace = True)
    x_test = pre.clean_value(x_test,np.nan,0,inplace = True)
    
    
    x_train = pre.build_poly(x_train, degre_polys[i])
    x_test = pre.build_poly(x_test,degre_polys[i])
    x_train = np.c_[np.ones((x_train.shape[0], 1)), x_train]
    x_test = np.c_[np.ones((x_test.shape[0], 1)), x_test]
    
    clean_data_trains.append((x_train,y_train))
    clean_data_tests.append((x_test,test_indx))

## Data Processing

_We now need to standardize the function so that they all take the same type of parameters as inputs_

In [ ]:
init_w = np.random.rand(clean_data_tests[0][0].shape[1])
maxiters = 100
gamma = 0.01

#method 1
meth1 = lambda  y, x: lst.ridge_regression(y,x,5.17E-5)

if (len(clean_data_tests) > 1):
    init_w2 = np.random.rand(clean_data_tests[1][0].shape[1])
    #method 2
    #reg_log_reg = lambda y,x : log.reg_logistic_regression(y, x, lambda_, init_w2, maxiters, gamma)
    #meth2 = lambda  y, x: log.logistic_regression(y,x,init_w2,5,gamma)
    meth2 = lambda y,x : lst.ridge_regression(y,x,0.0013)

    init_w3 = np.random.rand(clean_data_tests[2][0].shape[1])
    #method 3
    lambda_ = 0.1
    meth3 = lambda y, x: lst.ridge_regression(y,x,0.001389)
    #log_reg3 = lambda  y, x: log.logistic_regression(y,x,init_w3,5,gamma)

methods = [meth1,meth2,meth3]

_At this point we try the different models defined in the cell above: to do so run the cell below, and check the obtained accuracies._

In [ ]:
accuracies_group_means = []
accuracies_group_stds = []
for round_,((x_train,y_train),meth) in enumerate(zip(clean_data_trains,methods)):
    print("#################################")
    print("**********treating the {i}th group of data:**************".format(i = round_+1))
    acc_mean, acc_std = spl.k_fold_cv(y_train,x_train,5,meth)
    accuracies_group_means.append(acc_mean)
    accuracies_group_stds.append(acc_std)
print("done! Obtained accuracies:",[np.mean(i) for i in accuracies_group_means])

## Optimization

#### Ridge

In [ ]:
# for ridge: for every models test different lambdas and degrees
degrees = np.arange(1, 3)
lambdas = np.logspace(-5, 0, 2)

In [ ]:
accuracies_tot = []

for idx_subset, (x_train, y_train) in enumerate(clean_data_trains):
    print('##### START SUBSET {} #####'.format(idx_subset))
    accuracies = np.zeros((len(lambdas), len(degrees)))
    for idx_deg, deg in enumerate(degrees):
        x_poly = pre.build_poly(x_train, deg)
        
        for idx_lambda, lambda_ in enumerate(lambdas):
            ridge = lambda y, x: lst.ridge_regression(y,x,lambda_)
            _, k_accuracies_test = spl.k_fold_cv(y_train, x_poly, 2, ridge)
            
            # update table
            accuracies[idx_lambda][idx_deg] = np.mean(k_accuracies_test)
    
    accuracies_tot.append(accuracies)
    print('##### END SUBSET {} #####'.format(idx_subset))


In [ ]:
# save accuracies
import pickle

with open('acc_ridge.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(accuracies_tot, f)

In [ ]:
import seaborn as sns

plt.subplots(figsize=(15,5))
plt.subplot(1,3,1)
fig = sns.heatmap(accuracies_tot[0])
fig.set_yticklabels(np.round(lambdas, 5), rotation=60)
fig.set_xticklabels(degrees)
fig.set_xlabel('degree')
fig.set_ylabel('lambda')
fig.set_title('Accuracy - subset 0')
plt.subplot(1,3,2)
fig = sns.heatmap(accuracies_tot[1])
fig.set_xticklabels(degrees)
fig.set_xlabel('degree')
fig.set_ylabel('lambda')
fig.set_title('Accuracy - subset 1')
plt.subplot(1,3,3)
fig = sns.heatmap(accuracies_tot[2])
fig.set_xticklabels(degrees)
fig.set_xlabel('degree')
fig.set_ylabel('lambda')
fig.set_title('Accuracy - subset 2')

In [ ]:
# plt version
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax0, ax1, ax2 = axes.flatten()

d0 = ax0.imshow(accuracies_tot[0], cmap='GnBu', aspect='auto')
ax0.set_yticklabels(np.round(lambdas, 5), rotation=60)
plt.xticks(degrees)
ax0.set_xlabel('degree', fontsize=12)
ax0.set_ylabel('lambda', fontsize=12)
ax0.set_title('Subset 0', fontsize=15)
fig.colorbar(d0, ax=ax0)

d1 = ax1.imshow(accuracies_tot[1], cmap='GnBu', aspect='auto')
ax1.set_yticklabels(np.round(lambdas, 5), rotation=60)
ax1.set_xlabel('degree', fontsize=12)
ax1.set_ylabel('')
ax1.set_title('Subset 1', fontsize=15)
fig.colorbar(d1, ax=ax1)

d2 = ax2.imshow(accuracies_tot[2], cmap='GnBu', aspect='auto')
ax2.set_yticklabels(np.round(lambdas, 5), rotation=60)
#plt.xticks(degrees)
ax2.set_xlabel('degree', fontsize=12)
ax2.set_ylabel('')
ax2.set_title('Subset 2', fontsize=15)
fig.colorbar(d2, ax=ax2)

plt.subplots_adjust(wspace=0.3)

In [ ]:
#Save figure
fig.savefig("heatmaps.png")

In [ ]:
# y: ↓ (lambdas), x: → (degree)
for nb, acc in enumerate(accuracies_tot):
    print('SUBSET {}'.format(nb))
    ymax = np.asscalar(np.where(acc == np.max(acc))[0])
    xmax = np.asscalar(np.where(acc == np.max(acc))[1])
    
    print('Best degree for subset {}: {}'.format(nb, degrees[xmax]))
    print('Best lambda for subset {}: {}'.format(nb, lambdas[ymax]))
    print('Accuracy: {}'.format(acc[ymax][xmax]))

## Visualization

In [ ]:
lambdas = np.logspace(-5, 0, 10)
plt.figure(1, figsize=(16, 16))
for idx_subset, (x_train, y_train) in enumerate(clean_data_trains):
    accuracy_train = []
    accuracy_test = []
    
    for idx_lambda, lambda_ in enumerate(lambdas):
            ridge = lambda y, x: lst.ridge_regression(y,x,lambda_)
            
            k_accuracies_train, k_accuracies_test = spl.k_fold_cv(y_train, x_train, 10, ridge, hlp.accuracy)
            
            plt.subplot(3, 3, idx_subset+1)
            plt.title("Cross validation for training of categorical subset {i}".format(i=idx_subset))
            viz.visualize_boxplot_cross_validation(k_accuracies_train, [idx_lambda])
            
            plt.subplot(3, 3, 4+idx_subset)
            plt.title("Cross validation for testing of categorical subset {i}".format(i=idx_subset))
            viz.visualize_boxplot_cross_validation(k_accuracies_test, [idx_lambda])
        
            accuracy_train.append(np.mean(k_accuracies_train))
            accuracy_test.append(np.mean(k_accuracies_test))
            
    plt.subplot(3, 3, idx_subset+7)
    plt.title("Cross validation of subset {i}".format(i=idx_subset))
    viz.cross_validation_visualization(lambdas, accuracy_train, accuracy_test)
    

## Submission

_We now interpolate the data thanks to the model defined 2 cells higher..._

In [ ]:
y_submit = np.zeros(len(tX_test))
assert(len(tX_test) == sum([i[0].shape[0] for i in clean_data_tests]))
for (x_test,y_indx),(x_train,y_train),meth in zip(clean_data_tests,clean_data_trains,methods):
    w_fin,loss = meth(y_train,x_train)
    y_test = x_test @ w_fin
    y_test = [-1 if i < 0 else 1.0 for i in y_test]
    y_submit[y_indx] = y_test

_And finally save the results to csv._

In [ ]:
hlp.create_csv_submission(ids_test,y_submit,"anakin.csv")

## Put your useful trash here

In [ ]:
#trash random dataframe
np.random.seed(2)
df = pd.DataFrame(np.random.randint(-1002,-995,size =(3,4)), columns=list('ABCD'))
df.replace(to_replace = -999,value = np.nan)

In [ ]:
#Yann
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)